# nn-UNet v2 Example
Masoom Haider, MD, FRCPC, FSAR, FCAHS  
Director AIMS Lab, Sinai Health System  
Joint Dept of Medical Imaging, Univeristy of Toronto  
\
\
GitHub Library for nnUNet  
https://github.com/MIC-DKFZ/nnUNet  
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring 
method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.

This example uses some of the data from PI-CAI  
https://pi-cai.grand-challenge.org/

## Pre-requisites
- Familiarity with Python coding - intermediate to expert level
- Jupyter notebook installed in python environment


## Installation/Requirements
- See https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/installation_instructions.md  
- Will work on a single GPU workstation/gaming system i.e NVidia 4090 24G VRAM for abdo CT and MRI ($3-5K) 2025
- GPU support for pytorch install


In [ ]:
# install nnUNetv2
# %pip install nnunetv2

## Preparing Data
https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/dataset_format.md

### DICOM to NIfTI
- Many open source AI computer vision models don't directly support DICOM
- Use the NIfTI format (Neuroimaging Informatics Technology Initiative)based requiring DICOM to NIfTI conversion
- Watch out for series naming, mutiphase in one series, patched image volumes
- Preparing and checking series can be time consuming and can benefit from AI tools for series extraction (research)
- Watch out
  - data compression schemes
  - volume concantenation (multiphase, broken between chest , abdomen, pelvis)
  - LAS-DICOM
  - RAI (NIfTI)
  - supine vs prone
  - embedded localizers

**Python Libraries or Viewer Conversion - Not covered in this example**
- **SimpleITK**
- nibabel https://nipy.org/nibabel/
- dicom2nifti https://github.com/icometrix/dicom2nifti
- 3D Slicer https://www.slicer.org/


### Sorting Data and Folder Structure
Data and models are stored in a directory structure defined my some environment variables which must be setup before training and prediction  
Same convention is used for training and prediction

- MLDIRNAME
  - nnUNet_raw
      - Dataset001_BrainTumour *(this is the name of the model)*
      - Dataset002_Heart
      - Dataset003_Liver
      - Dataset004_Hippocampus
      - Dataset005_Prostate
        - dataset.json
        - imagesTr
          - I_001_0000.nii.gz *(channel 0 = axt2)*
          - I_001_0001.nii.gz *(channel 1 = adc)*
          - I_001_0002.nii.gz *(channel 2 = b1600)*
          - ...
        - imagesTs
        - labelsTr *(one label=segmentatotin masks regardless of number of channels)*
          - I_001.nii.gz
          - I_002.nii.gz
          - ...

In [ ]:
import os
from utils.nnunetutils import fInitialize

mldatadir = os.path.join(os.environ('MLDATADIR'),'NNUNETTALKDATA')  # replace with your ML data path in this variable
# You can put the PICAI directory in the repo in the mldatadir

rootdir = os.path.join(mldatadir, 'NNUNET')
modelname = 'Dataset100_prostate'
dirdic = fInitialize(rootdir, modelname)  # creates directory structure and returns dictionary of dir locations

In [ ]:
from utils.nnunetutils import fPrepareFilesMP

# where the PICAI data is located
sourcedir = os.path.join(mldatadir, "PICAI")

labels = {
    'background': 0,
    'tumor': 1
}

region_class_order = [1]
imagefilenames = ["axt2.nii.gz", "adc-axt2.nii.gz","b1600-axt2.nii.gz"]
segfilename = "t1_axt2_he.nii.gz"
channel_names = {0: "axt2", 1:"adc", 2:"b1600"}  

fPrepareFilesMP(dirdic, channel_names, labels, sourcedir, imagefilenames, segfilename, region_class_order)


## Preprocess the Images
Determine imaging parameters to create an optimal UNet model architecture  

https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/assets/nnU-Net_overview.png

![alt text](image.png)

In [ ]:
from utils.nnunetutils import fPreProcess
fPreProcess(modelname)

## Train the Model

In [5]:
from utils.nnunetutils import fTrain
configurations = ['3d_fullres'] #['2d', '3d_fullres', '3d_lowres', '3d_cascade_fullres']
folds = [1]
fTrain(modelname, configurations=configurations, folds=folds)

## Predict Using Model

In [ ]:
import os
from utils.nnunetutils import fPredict

mldatadir = os.environ('MLDATADIR')  # put your ML data path in this variable
rootdir = os.path.join(mldatadir, )
modelname = 'Dataset100_prostate'

# folder with case yuo wish to predict
mFolder = os.path.join(mldatadir, r'PICAITEST\AIPR_10005_20120718_1000005')  

filestopredict  = ["axt2.nii.gz", "adc-axt2.nii.gz","b1600-axt2.nii.gz"]
sourcefnames = [os.path.join(mFolder, f) for f in filestopredict]
segfilename = os.path.join(mFolder,"t1_axt2_nnunet-100.nii.gz")
configuration = '3d_fullres'
fPredict(rootdir, modelname, sourcefnames, segfilename, configuration)
            